In [ ]:
import logging
from pathlib import Path
import polars as pl
import altair as alt

from analyze.loader import DeviceDataSource
from analyze.logger import POLAR_ANALYZER_LOGGER
from analyze.model import PolarDeviceData
from analyze.common import PHASE_COLUMNS, Phase
from config import config

logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(threadName)s - %(levelname)s - %(name)s - %(message)s")
POLAR_ANALYZER_LOGGER.setLevel(logging.INFO)
logger = POLAR_ANALYZER_LOGGER.getChild("notebook")

In [ ]:
data = PolarDeviceData.load([DeviceDataSource(f.dir, f.device) for f in config.files])

In [ ]:
logger.info("Collecting data...")
df = data.phase_data.collect()
logger.info(f"Found {len(df)} rows")
df.head(3)

## Energy by day of week

In [42]:
week_day = data.total_energy_by_day_of_week()
week_day = week_day.with_columns(pl.concat_str([pl.col("device"), pl.col("phase")], separator="_").alias("device_phase"))
week_day = week_day.collect()
week_day

day_of_week_num,device,phase,total_act_energy_kwh,day_of_week,device_phase
i8,str,enum,f64,enum,str
1,"""unten""","""c""",1.291304,"""Monday""","""unten_c"""
1,"""oben""","""c""",0.10834,"""Monday""","""oben_c"""
1,"""unten""","""b""",0.703121,"""Monday""","""unten_b"""
1,"""oben""","""b""",0.156884,"""Monday""","""oben_b"""
1,"""unten""","""a""",0.748198,"""Monday""","""unten_a"""
…,…,…,…,…,…
7,"""unten""","""c""",1.491672,"""Sunday""","""unten_c"""
7,"""unten""","""b""",0.691111,"""Sunday""","""unten_b"""
7,"""oben""","""c""",0.097979,"""Sunday""","""oben_c"""


In [94]:
week_day_chart = alt.Chart(week_day).mark_bar(size=15, tooltip=True).encode(
            x=alt.X(field="day_of_week", type="nominal").title("Day of week").sort({"field": "day_of_week_num"}),
            y=alt.Y(field="total_act_energy_kwh", type="quantitative").title("Energy [kWh]"),
            color=alt.Color(field="device_phase", type="nominal").title("Device / Phase"),
            order=alt.Order(field="device_phase", type="nominal")
        ).properties(title="Energy by day of week")
week_day_chart

alt.Chart(...)

## Energy by Month

In [76]:
month = data.total_energy(every="1mo")
month = month.with_columns(pl.concat_str([pl.col("device"), pl.col("phase")], separator="_").alias("device_phase"))
month = month.collect()
month

device,phase,date,total_act_energy_kwh,device_phase
str,enum,date,f64,str
"""unten""","""a""",2024-05-01,13.656692,"""unten_a"""
"""unten""","""a""",2024-06-01,20.890274,"""unten_a"""
"""unten""","""a""",2024-07-01,24.905331,"""unten_a"""
"""unten""","""a""",2024-08-01,25.656352,"""unten_a"""
"""unten""","""a""",2024-09-01,27.961295,"""unten_a"""
…,…,…,…,…
"""oben""","""c""",2024-07-01,2.997995,"""oben_c"""
"""oben""","""c""",2024-08-01,2.980132,"""oben_c"""
"""oben""","""c""",2024-09-01,2.857883,"""oben_c"""


In [95]:
chart = (
    alt.Chart(month)
    .mark_bar(size=20,tooltip=True)
    .encode(
        x=alt.X("date", type="temporal").title("Date"),
        y=alt.Y("total_act_energy_kwh", type="quantitative").title("Energy [kWh]"),
        color=alt.Color("device_phase", type="nominal").title("Device / Phase"),
    )
    .properties(title="Energy by month")
)
chart

alt.Chart(...)

## Energy by week

In [56]:
week = data.total_energy(every="1w")
week = week.with_columns(pl.concat_str([pl.col("device"), pl.col("phase")], separator="_").alias("device_phase"))
week = week.collect()
week

device,phase,date,total_act_energy_kwh,device_phase
str,enum,date,f64,str
"""unten""","""a""",2024-05-13,4.520481,"""unten_a"""
"""unten""","""a""",2024-05-20,5.397368,"""unten_a"""
"""unten""","""a""",2024-05-27,5.173986,"""unten_a"""
"""unten""","""a""",2024-06-03,3.714145,"""unten_a"""
"""unten""","""a""",2024-06-10,5.505918,"""unten_a"""
…,…,…,…,…
"""oben""","""c""",2024-10-21,0.652337,"""oben_c"""
"""oben""","""c""",2024-10-28,0.745624,"""oben_c"""
"""oben""","""c""",2024-11-04,0.740028,"""oben_c"""


In [57]:
df.plot.line(
            x=alt.X("date", type="temporal").title("Date"),
            y=alt.Y("total_act_energy_kwh", type="quantitative").title("Energy [kWh]"),
            color=alt.Color("device_phase", type="nominal").title("Device / Phase"),
        ).properties(title="Energy by week")

MaxRowsError: The number of rows in your dataset is greater than the maximum allowed (5000).

Try enabling the VegaFusion data transformer which raises this limit by pre-evaluating data
transformations in Python.
    >> import altair as alt
    >> alt.data_transformers.enable("vegafusion")

Or, see https://altair-viz.github.io/user_guide/large_datasets.html for additional information
on how to plot large datasets.

alt.Chart(...)

In [ ]:
data.plot_total_energy("1d")